## 1. Analysis of costs to run animal shelters
<p>You are working as a data analyst in the animal welfare department. In preparation for next year's budget, the head of your department would like to know the total cost to shelter animals, broken down by each animal type and size combination (i.e., 'small dogs', 'large dogs', etc.). You need to prepare a table that contains this information.</p>
<p>The total cost to shelter an animal for the year is calculated as the sum of three factors: the size and animal type, the age, and the location.</p>
<ul>
<li>The base cost of sheltering an animal is based upon its size and type. The costs per animal type and size are contained in the <code>size_costs</code> table. The criteria for classifying size has recently been updated, and so you will need to categorize animals based upon the following table:</li>
</ul>
<table>
<thead>
<tr>
<th style="text-align:right;"></th>
<th style="text-align:right;">Small</th>
<th>Medium</th>
<th>Large</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:right;">Dog</td>
<td style="text-align:right;">&lt;= 10 lbs</td>
<td>10 lbs &lt; and &lt;= 30 lbs</td>
<td>30 lbs &lt;</td>
</tr>
<tr>
<td style="text-align:right;">Cat</td>
<td style="text-align:right;">&lt;= 5 lbs</td>
<td>5 lbs &lt; and &lt;= 7 lbs</td>
<td>7 lbs &lt;</td>
</tr>
<tr>
<td style="text-align:right;">Bird</td>
<td style="text-align:right;">&lt;= 0.7 lbs</td>
<td>0.7 lbs &lt; and &lt;= 1.1 lbs</td>
<td>1.1 lbs &lt;</td>
</tr>
</tbody>
</table>
<ul>
<li>Older animals cost more, and so an age cost (contained in the <code>age_costs</code> table) is added. Each animal's age should be calculated as the age by the end of the year (December 31st, 2021).</li>
<li>There is a location cost depending on where the animal is sheltered (contained in the <code>location_costs</code> table). </li>
<li>The calculation should not include animals that have been sponsored by private charities (sponsored animals are listed in the <code>sponsored_animals</code> table).</li>
</ul>
<p>For future visualization purposes, you will also need to include a <code>percentage</code> column in your result. This percentage should reflect the fraction of the total cost to be allocated to each animal and size combination.</p>
<p>The data you need is available in the tables shown in the database schema below.</p>
<h5 id="databaseschema">Database Schema</h5>
<p><img src="https://assets.datacamp.com/production/repositories/5934/datasets/a946a159c0024ee0995f7a030f2c0cf802203835/diagram.PNG" width="500" height="500"> </p>

In [2]:
%%sql
postgresql:///animal_shelters
ALTER TABLE animals 
ALTER COLUMN birthdate TYPE date
USING birthdate::timestamp;
        
ALTER TABLE age_costs
ALTER COLUMN age TYPE INT
USING age::integer

Done.
Done.


[]

In [3]:
%%sql
postgresql:///animal_shelters
CREATE VIEW animals_info AS
    SELECT
    animalid,
    animaltype,
    weight,
    location,
    CAST(DATE_PART('year', AGE('2021-12-31'::timestamp, birthdate)) AS INTEGER) AS age,
    CASE  
        WHEN animaltype = 'Dog' AND weight <= 10 THEN 'DS'
        WHEN animaltype = 'Dog' AND weight > 10 AND weight <= 30 THEN 'DM'
        WHEN animaltype = 'Dog' AND weight > 30 THEN 'DL'
        WHEN animaltype = 'Cat' AND weight <= 5 THEN 'CS'
        WHEN animaltype = 'Cat' AND weight > 5 AND weight <= 7 THEN 'CM'
        WHEN animaltype = 'Cat' AND weight > 7 THEN 'CL'
        WHEN animaltype = 'Bird' AND weight <= 0.7 THEN 'BS'
        WHEN animaltype = 'Bird' AND weight > 0.7 AND weight <= 1.1 THEN 'BM'
        WHEN animaltype = 'Bird' AND weight > 1.1 THEN 'BL'
    END AS sizeid
    FROM animals

Done.


[]

In [4]:
%%sql
postgresql:///animal_shelters    
SELECT *
FROM animals_info
LIMIT(10)

10 rows affected.


animalid,animaltype,weight,location,age,sizeid
80066a4b41ac06,Cat,4.0,Fort Worth,2,CS
968bf2e7230d2f,Cat,7.0,Houston,9,CM
8aacfe19c56b0f,Dog,45.0,Austin,8,DL
7126d96889803f,Cat,3.0,Houston,6,CS
9617240ad5b268,Cat,9.0,Houston,7,CL
fd7bf1fa986957,Dog,39.0,Austin,4,DL
691081c83b1075,Dog,3.0,Fort Worth,5,DS
f957d0fabf9e63,Dog,24.0,Fort Worth,3,DM
43dfa7ef83d83b,Dog,36.0,San Antonio,3,DL
2f17d474be934d,Dog,16.0,Austin,4,DM


In [5]:
%%sql
postgresql:///animal_shelters
CREATE TABLE animals_info_ns AS
    SELECT
    animals_info.animalid,
    animals_info.sizeid,
    animals_info.animaltype,
    st.size,
    lt.costs + st.costs + at.costs AS total_per_animal
    FROM animals_info 
    JOIN location_costs lt ON lt.location = animals_info.location
    JOIN size_costs st ON st.sizeid = animals_info.sizeid
    JOIN age_costs at ON at.age = animals_info.age
    WHERE animals_info.animalid NOT IN (
        SELECT sponsorid
        FROM sponsored_animals)

9757 rows affected.


[]

In [6]:
%%sql
postgresql:///animal_shelters
SELECT SUM(total_per_animal)
FROM animals_info_ns

1 rows affected.


sum
3477015


In [7]:
%%sql
postgresql:///animal_shelters
SELECT
LOWER(animaltype) AS animaltype,
LOWER(size) AS size,
SUM(total_per_animal) AS total,
ROUND(100*(SUM(total_per_animal)/3419935.0), 2) AS percentage
FROM animals_info_ns
GROUP BY animaltype, size
ORDER BY animaltype, size DESC

9 rows affected.


animaltype,size,total,percentage
bird,small,1615,0.05
bird,medium,3460,0.10
bird,large,7770,0.23
cat,small,518015,15.15
cat,medium,250575,7.33
cat,large,439490,12.85
dog,small,336530,9.84
dog,medium,941895,27.54
dog,large,977665,28.59
